In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
import time
import datetime
from tensorflow.keras.applications.mobilenet import preprocess_input
import json
import cv2
import glob

from torch.optim.optimizer import Optimizer, required

from torch.autograd import Variable
from torch import nn
import torch
from torch import Tensor
from torch.nn import Parameter
import tarfile

import torchvision      
from torchvision.utils import save_image
import torchvision.transforms as transforms

import os
print(os.listdir("../input/"))


In [ ]:
df = pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/leaf.csv")
df.to_csv("leaf.csv")
print(len(df))
df.head()

# 处理数据

In [ ]:

def draw_cv2(raw_strokes, size=256, lw=6, time_color=True,BASE_SIZE = 256):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def image_generator_xd(size, batchsize = 64, lw=6, n = 0,
                       time_color=True, NCATS = 340,DP_DIR = '../input/shuffle-csvs/'):
    paths = glob.glob(DP_DIR + "*.csv*")
    paths.sort()
    paths = paths[n:n+1]
    paths = ['../input/quickdraw-doodle-recognition/train_simplified/leaf.csv']
    print(paths)
    while True:
        for filename in paths:
            for df in pd.read_csv(filename, chunksize=batchsize,engine='python'):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
#                 y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                y = 0
                yield x, y
   

In [ ]:

train_datagen = image_generator_xd(size=64, batchsize=128, n = 221,
                                  DP_DIR = '../input/quickdraw-doodle-recognition/train_simplified/')
x, y = next(train_datagen)
# display_img(20,x)


In [ ]:
def display_img(n,img):
    if isinstance(img,torch.Tensor):
        img = [np.rollaxis(i.numpy(), 0, 3)  for i in img]
    for i in range(n):
        plt.subplot(2,n//2,i+1)
        plt.imshow(img[i])
        plt.axis('off')
display_img(20,x)
plt.show()

# 建立模型

In [ ]:

def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)

class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False


    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)


    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)

In [ ]:


class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim,activation):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.activation = activation
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out,attention

class Generator(nn.Module):
    """Generator."""

    def __init__(self, img_dim = 3, image_size=64, z_dim=100, conv_dim=64):
        super(Generator, self).__init__()
        self.imsize = image_size
        layer1 = []
        layer2 = []
        layer3 = []
        last = []

        repeat_num = int(np.log2(self.imsize)) - 3
        mult = 2 ** repeat_num # 8
        layer1.append(SpectralNorm(nn.ConvTranspose2d(z_dim, conv_dim * mult, 4)))
        layer1.append(nn.BatchNorm2d(conv_dim * mult))
        layer1.append(nn.ReLU())

        curr_dim = conv_dim * mult

        layer2.append(SpectralNorm(nn.ConvTranspose2d(curr_dim, int(curr_dim / 2), 4, 2, 1)))
        layer2.append(nn.BatchNorm2d(int(curr_dim / 2)))
        layer2.append(nn.ReLU())

        curr_dim = int(curr_dim / 2)

        layer3.append(SpectralNorm(nn.ConvTranspose2d(curr_dim, int(curr_dim / 2), 4, 2, 1)))
        layer3.append(nn.BatchNorm2d(int(curr_dim / 2)))
        layer3.append(nn.ReLU())

        if self.imsize == 64:
            layer4 = []
            curr_dim = int(curr_dim / 2)
            layer4.append(SpectralNorm(nn.ConvTranspose2d(curr_dim, int(curr_dim / 2), 4, 2, 1)))
            layer4.append(nn.BatchNorm2d(int(curr_dim / 2)))
            layer4.append(nn.ReLU())
            self.l4 = nn.Sequential(*layer4)
            curr_dim = int(curr_dim / 2)

        self.l1 = nn.Sequential(*layer1)
        self.l2 = nn.Sequential(*layer2)
        self.l3 = nn.Sequential(*layer3)

        last.append(nn.ConvTranspose2d(curr_dim, img_dim, 4, 2, 1))
        last.append(nn.Tanh())
        self.last = nn.Sequential(*last)

        self.attn1 = Self_Attn( 128, 'relu')
        self.attn2 = Self_Attn( 64,  'relu')

    def forward(self, z):


        
        z = z.view(z.size(0), z.size(1), 1, 1) 
        # torch.Size([128, 128, 1, 1])
        out=self.l1(z) # torch.Size([64, 512, 4, 4])
        out=self.l2(out) # torch.Size([64, 256, 8, 8])
        out=self.l3(out) # torch.Size([64, 128, 16, 16])
        
        out,p1 = self.attn1(out) # torch.Size([64, 128, 16, 16])  torch.Size([64, 256, 256])
        
        out=self.l4(out) # torch.Size([64, 64, 32, 32])  
        
        out,p2 = self.attn2(out) # torch.Size([64, 64, 32, 32])  torch.Size([64, 1024, 1024])
        
        out=self.last(out) # torch.Size([64, 3, 64, 64])

        return out, p1, p2

In [ ]:
class Discriminator(nn.Module):
    """Discriminator, Auxiliary Classifier."""

    def __init__(self,  img_dim = 3,image_size=64, conv_dim=64):
        super(Discriminator, self).__init__()
        self.imsize = image_size
        layer1 = []
        layer2 = []
        layer3 = []
        last = []

        layer1.append(SpectralNorm(nn.Conv2d(img_dim, conv_dim, 4, 2, 1)))
        layer1.append(nn.LeakyReLU(0.1))

        curr_dim = conv_dim

        layer2.append(SpectralNorm(nn.Conv2d(curr_dim, curr_dim * 2, 4, 2, 1)))
        layer2.append(nn.LeakyReLU(0.1))
        curr_dim = curr_dim * 2

        layer3.append(SpectralNorm(nn.Conv2d(curr_dim, curr_dim * 2, 4, 2, 1)))
        layer3.append(nn.LeakyReLU(0.1))
        curr_dim = curr_dim * 2

        if self.imsize == 64:
            layer4 = []
            layer4.append(SpectralNorm(nn.Conv2d(curr_dim, curr_dim * 2, 4, 2, 1)))
            layer4.append(nn.LeakyReLU(0.1))
            self.l4 = nn.Sequential(*layer4)
            curr_dim = curr_dim*2
        self.l1 = nn.Sequential(*layer1)
        self.l2 = nn.Sequential(*layer2)
        self.l3 = nn.Sequential(*layer3)

        last.append(nn.Conv2d(curr_dim, 1, 4))
        self.last = nn.Sequential(*last)

        self.attn1 = Self_Attn(256, 'relu')
        self.attn2 = Self_Attn(512, 'relu')

    def forward(self, x):
#         torch.Size([128, 1, 64, 64])
        out = self.l1(x) # torch.Size([200, 64, 32, 32])
        out = self.l2(out) # torch.Size([200, 128, 16, 16])
        out = self.l3(out) # torch.Size([200, 256, 8, 8])
        
        out,p1 = self.attn1(out) # torch.Size([200, 256, 8, 8]) torch.Size([200, 64, 64])
        out=self.l4(out)# torch.Size([200, 512,4, 4])
        out,p2 = self.attn2(out) # torch.Size([200, 512, 4, 4]) torch.Size([200, 16, 16])
        
        out=self.last(out) # torch.Size([200, 1, 1, 1])
        

        return out.squeeze(), p1, p2

# 训练部分

In [ ]:
adv_loss='wgan-gp'
attn_path='./attn'
batch_size=64
beta1=0.0
beta2=0.9
d_conv_dim=64
d_iters=5
d_lr=0.0004
dataset='cifar'
g_conv_dim=64 
g_lr=0.0001
g_num=5
image_path='./data'
model_save_path='./models'
log_path='./logs'
sample_path='./samples'
imsize=64
lambda_gp=10
lr_decay=0.95
model='sagan'
num_workers=2
parallel=False
pretrained_model=None
is_Cifar = True
total_step=2000
log_step=total_step//100
sample_step=total_step//100
model_save_step=total_step//50

train=True
use_tensorboard=False
version='sagan_1'
z_dim=128

for i in [image_path,log_path,model_save_path,sample_path]:
    if not os.path.exists(i):
        os.mkdir(i)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp_(0, 1)

def tensor2var(x, grad=False):
    if not isinstance(x,torch.Tensor):
        x = torch.Tensor(x).unsqueeze(1)
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, requires_grad=grad)
!ls

In [ ]:
# G = Generator(imsize, z_dim, g_conv_dim).cuda()
# D = Discriminator(imsize, d_conv_dim).cuda()

# def load_modle(G,D,path = None):
#     if path is None:
#         modle_load_path = glob.glob('../input/sagan/models/*.pth')
#     modle_load_path.sort()
#     if len(modle_load_path)>2:

#         modle_load_path = modle_load_path[-2:]
#         print(modle_load_path)
#         D.load_state_dict(torch.load(modle_load_path[0]))
#         G.load_state_dict(torch.load(modle_load_path[1]))
#     else:
#         print('file not found')
#     return G,D
# G,D = load_modle(G,D,path = None)

# z = tensor2var(torch.randn(real_images.size(0), z_dim))

# fake_images,gf1,gf2 = G(z)
# d_out_fake,df1,df2 = D(fake_images)

# # z.shape, fake_images.shape, d_out_fake.shape
# # (torch.Size([64, 128]), torch.Size([64, 3, 64, 64]), torch.Size([64]))
# display_img(10,fake_images.detach().cpu())

模型生成与优化器选择

In [ ]:

is_Cifar = False
if is_Cifar:
    img_dim = 3
    dataloader = trainloader
else:
    img_dim = 1
    dataloader = train_datagen
    
G = Generator(img_dim = img_dim ,image_size=imsize, z_dim = z_dim, conv_dim=g_conv_dim).cuda()

D = Discriminator(img_dim = img_dim,image_size=imsize, conv_dim = d_conv_dim).cuda()
try:
    G,D = load_modle(G,D,path = None)
except:
    print("load error")

g_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, G.parameters()), g_lr, [beta1, beta2])
d_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), d_lr, [beta1, beta2])

c_loss = torch.nn.CrossEntropyLoss()

# print(G)
# print(D)


In [ ]:
# 设置训练次数

    
data_iter = iter(dataloader)
# Start with trained model
if pretrained_model:
    start = pretrained_model + 1
else:
    start = 0

# Fixed input for debugging 
fixed_z = tensor2var(torch.randn(batch_size, z_dim))
model_save_step

In [ ]:
# Start time
hist = []
start_time = time.time()
for step in range(start, total_step):

    # ================== Train D ================== #
    D.train()
    G.train()

    try:
        real_images, _ = next(data_iter)
    except:
        print("error")
        data_iter = iter(dataloader)
        real_images, _ = next(data_iter)

    # Compute loss with real images
    # dr1, dr2, df1, df2, gf1, gf2 are attention scores
    real_images = tensor2var(real_images)
    d_out_real,dr1,dr2 = D(real_images)
    if adv_loss == 'wgan-gp':
        d_loss_real = - torch.mean(d_out_real)
    elif adv_loss == 'hinge':
        d_loss_real = torch.nn.ReLU()(1.0 - d_out_real).mean()

    # apply Gumbel Softmax
    z = tensor2var(torch.randn(real_images.size(0), z_dim))
    fake_images,gf1,gf2 = G(z)
    d_out_fake,df1,df2 = D(fake_images)

    if adv_loss == 'wgan-gp':
        d_loss_fake = d_out_fake.mean()
    elif adv_loss == 'hinge':
        d_loss_fake = torch.nn.ReLU()(1.0 + d_out_fake).mean()


    # Backward + Optimize
    d_loss = d_loss_real + d_loss_fake
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()
    

    if adv_loss == 'wgan-gp':
        # Compute gradient penalty 
        alpha = torch.rand(real_images.size(0), 1, 1, 1).cuda().expand_as(real_images)
        interpolated = Variable(alpha * real_images.data + (1 - alpha) * fake_images.data,
                                requires_grad=True)
        out,_,_ = D(interpolated)

        grad = torch.autograd.grad(outputs=out,
                                   inputs=interpolated,
                                   grad_outputs=torch.ones(out.size()).cuda(),
                                   retain_graph=True,
                                   create_graph=True,
                                   only_inputs=True)[0]

        grad = grad.view(grad.size(0), -1)
        grad_l2norm = torch.sqrt(torch.sum(grad ** 2, dim=1))
        d_loss_gp = torch.mean((grad_l2norm - 1) ** 2)

        # Backward + Optimize
        d_loss = lambda_gp * d_loss_gp

        d_optimizer.zero_grad()
#         g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

    # ================== Train G and gumbel ================== #
    # Create random noise
    z = tensor2var(torch.randn(real_images.size(0), z_dim))
    fake_images,_,_ = G(z)

    # Compute loss with fake images
    g_out_fake,_,_ = D(fake_images)  # batch x n
    if adv_loss == 'wgan-gp':
        g_loss_fake = - g_out_fake.mean()
    elif adv_loss == 'hinge':
        g_loss_fake = - g_out_fake.mean()

#     d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    g_loss_fake.backward()
    g_optimizer.step()


    # Print out log info
    if (step + 1) % log_step == 0:
        elapsed = time.time() - start_time
        elapsed = str(datetime.timedelta(seconds=elapsed))
        print("Elapsed [{}], G_step [{}/{}], D_step[{}/{}], d_out_real: {:.4f}, "
              " ave_gamma_l3: {:.4f}, ave_gamma_l4: {:.4f}".
              format(elapsed, step + 1, total_step, (step + 1),
                     total_step , d_loss_real.data,
                     G.attn1.gamma.mean().data, G.attn2.gamma.mean().data ))

    # Sample images
    if (step + 1) % sample_step == 0:
        print(str(step+1) + "  |  Sample Saved..")
        fake_images,_,_= G(fixed_z)
        save_image(denorm(fake_images.data),
                   os.path.join(sample_path, '{:04}_fake.png'.format(step + 1)))

    if (step+1) % model_save_step==0:
        print(str(step+1) + "  |  model Saved..")
        torch.save(G.state_dict(),
                   os.path.join(model_save_path, '{:05}_G.pth'.format(step + 1)))
        torch.save(D.state_dict(),
                   os.path.join(model_save_path, '{:05}_D.pth'.format(step + 1)))
   

    hist.append([step,float(d_loss.data.cpu().numpy()),float(g_loss_fake.data.cpu().numpy())])

# 绘制learning curve

In [ ]:
hist_ = np.array(hist)
fig, axs = plt.subplots(2,1, figsize=(13,7))

plt.subplot(2,1,1)
plt.plot(hist_[:,1])
plt.xlabel("step")
plt.ylabel("D loss")

plt.subplot(2,1,2)
plt.plot(hist_[:,2])
plt.xlabel("step")
plt.ylabel("G loss")

plt.show()

In [ ]:
im_paths = os.listdir("./samples/")

idx = [int(i[:-9]) for i in im_paths]
idx.sort(reverse=True)
best_idx = idx[:6]

best_idx,len(im_paths)

In [ ]:
fig, axs = plt.subplots(2,len(best_idx)//2, figsize=(13,9))
for i,idx in enumerate(best_idx):
    plt.subplot(2,len(best_idx)//2,i+1)
    a = plt.imread("./samples/" + "{:04}".format(idx) + "_fake.png")

    plt.imshow(a)
    plt.axis('off')